In [1]:
# importing dependencies
import pandas as pd
import numpy as np
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
import keras
import re
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize 
import emoji

Using TensorFlow backend.


In [6]:
# #stop word removal
from nltk.corpus import stopwords
stop={'حتى','مثل','على','أ','من' ,'أو','في حين', 'في', 'من قبل', 'ل','مع','حول', 'ضد','من','بين', 'إلى', 'من خلال', 'خلال', 'قبل', 'بعد', 'فوق', 'أدناه',' من ',' حتى ',' أسفل ',' في ',' خارج ',' على ',' قبالة ',' فوق ',' تحت ','مرة أخرى',' زيادة ',' ثم ' , 'مرة', 'هنا','هناك','عندما','حيث', 'لماذا', 'كيف', 'كل', 'أي', 'كلا', 'كل', 'قليل', 'أكثر', 'أكثر', 'الآخر', 'بعض', 'فقط ',' الخاصة ',' نفس ',' حتى ',' من ',' جدا ',' جدا ',' سوف ',' فقط ',' يجب ',' الآن '} #add related stop word here
print(stop)

{'أدناه', 'مرة', ' في ', 'من قبل', 'في حين', 'حول', ' على ', 'كيف', 'لماذا', 'كلا', 'هناك', 'فقط ', ' أسفل ', ' فوق ', ' خارج ', 'عندما', 'بعض', 'هنا', 'إلى', 'حيث', 'الآخر', ' ثم ', 'كل', 'بين', 'على', 'خلال', 'فوق', 'مثل', 'في', ' قبالة ', 'مرة أخرى', 'من', 'مع', 'من خلال', 'قليل', 'قبل', ' زيادة ', ' الخاصة ', ' نفس ', 'ضد', ' فقط ', ' حتى ', 'أي', ' من ', 'حتى', ' الآن ', ' يجب ', 'أ', ' جدا ', 'بعد', 'ل', ' تحت ', 'أو', ' سوف ', 'أكثر'}


In [7]:
df_csv = pd.read_csv('Arabic/offenseval-ar-training-v1.tsv', sep='\t')
df_test = pd.read_csv('Arabic/offenseval-ar-test-tweets-v1.tsv',sep='\t')

In [8]:
df_test.head(20001)

,id,tweet
0,8001,@USER اما انت تقعد طول عمرك لا مبدا ولا راي ثا...
1,8002,@USER @USER بتخاف نسوانك يزعلوا ولا ايه 😂 اه ي...
2,8003,RT @USER: يا عـسانـى نـبـقى يا عـمري حـبايـب و...
3,8004,RT @USER: باقي البيان وينو ما شفنه يا برهان <L...
4,8005,@USER @USER اللهم انت الشافي المعافي اشفيه وجم...
...,...,...
1995,9996,RT @USER: الله لايوفقك يا مهند عسيري يا معوق و...
1996,9997,RT @USER: @USER حبيبي يا يوسف وانت طيب يا صاحب...
1997,9998,RT @USER: يا بو محمد عشت يا طيب الفال<LF>عاشت ...
1998,9999,أنا مستني الحلقة بقالي سنتين يا بضان يا ابن ال...


In [9]:
#marge all data from files that has been read
twee=[]
ids=[]
for i in range(len(df_test)):
    ids.append(df_test['id'][i])
    twee.append(df_test['tweet'][i])

#marge all data from files that has been read
q_a=[]
label=[]
for i in range(len(df_csv)):
    label.append(df_csv['subtask_a'][i])
    q_a.append(df_csv['tweet'][i])
len(df_test)

2000

In [10]:
print(len(q_a))
print(len(label))
print(len(twee))
print(len(ids))

7000
7000
2000
2000


In [11]:
# convert labeling to 0 and 1
lb=[]
for i in label:
    if i=='OFF':
        lb.append(1)
    if i=='NOT':
        lb.append(0)

print(lb)
label=lb

[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [12]:
# remove empty space from the begining of a sentance
QA=[]
for i in q_a:
    sen=i.replace('@USER','')
    sen=sen.replace('@user','')
    sen=sen.replace("#", ' ')
    sen=sen.replace("<LF>", ' ')
    sen=sen.replace(":", ' ')
    sen=sen.replace("GoT", ' ')
    sen=sen.replace("URL", ' ')
    sen=sen.replace("RT", ' ') 
    sen=emoji.demojize(sen)
    sen=" ".join(sen.split())
    QA.append(sen)
q_a=QA

In [63]:
from string import ascii_lowercase
TWEETS=[]
for i in twee:
    sen=i.replace('@USER','')
    sen=sen.replace('@user','')
    sen=sen.replace("#", ' ')
    sen=sen.replace("<LF>", ' ')
    sen=sen.replace(":", ' ')
    sen=sen.replace("GoT", ' ')
    sen=sen.replace("URL", ' ')
    sen=sen.replace("RT", ' ') 
    sen=emoji.demojize(sen)
    for i in ascii_lowercase :
        sen= sen.replace(i, "")
    sen=" ".join(sen.split())
    TWEETS.append(sen)
twee=TWEETS

In [64]:
for i in twee:
    print(i)

اما انت تقعد طول عمرك لا مبدا ولا راي ثابت يا صعلوك . اقسم بربي ماتجي حاجة يا داعشي يا عميل الانكليز . راس مالك كلام بس . عقاب جبان . معمر سيد وحفتر راح يركب عليك .
بتخاف نسوانك يزعلوا ولا ايه :____: اه يا هلفوت يا بتاع الورد :____:
يا عـسانـى نـبـقى يا عـمري حـبايـب وحـبنـا يكبر معانا :_:.
باقي البيان وينو ما شفنه يا برهان ورينا يا برهان ورينا شو بيحصل في شلة الكيزان سلم بشير العار للحاكم السجان وباقي الكل…
اللهم انت الشافي المعافي اشفيه وجميع مرضى المسلمين والمسلمات شفاء لا يغادر سقما واصبغ عليهم عافيتك يا شافي يا معافى يا ارحم الراحمين.
آآآآآآآآآآآآآه يا غيث يا حبيبنا لو تعرف حجم الفراغ الي تركته فينا الله يرحمك و يغفرلك و يجمعنا بيك بعد العمر المديد في الفردوس…
يا حُب يا دُنيا جديدة يا احلى يا احلى ابيات القصيدة:_:.
صباح_الاربعاء . هذي السنين ستنقضي وستفرحين وستضحكين وتسعدين وتنعمين ستودعين الحزن يا روح اصبري فالله يا روحي " يحب الص…
عاوز اوجه رسالة ل الدكاترة و اني بعرف امارس اللغة مش المادة يا دحيحة يا بتاعة 99,5 في ثانوية عامة
حبيبي يا هندسه وانت طيب يا ابا :_:
الدورة_الشهرية يا

In [33]:
# #this code will stem text
# stem_both_q_a=[]
# for i in q_a:
#     ps = PorterStemmer() #stemming word removing
#     words = str(i).split(' ') 
#     s=''
#     for w in words: 
#         stem=ps.stem(w)
#         s=s+' '+stem
#     s=s.strip()

#     stem_both_q_a.append(s)
# q_a=stem_both_q_a

# print(q_a)

In [13]:
# # Read data files
comments_attack=pd.DataFrame()
comments_attack['comment']=q_a
comments_attack['label']=label

In [14]:
dataframe = comments_attack

In [15]:
# Text preprocessing
from string import ascii_lowercase
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace('@USER',''))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace("NEWLINE_TOKEN", " "))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace("TAB_TOKEN", " "))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(":", ""))
dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace("_", ""))
dataframe['comment'] = dataframe['comment'].apply((lambda x: re.sub('[/(){}\[\]\|@,;]','',x)))
for i in ascii_lowercase :
    dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(i, ""))



In [16]:
# this code will remove stop word
for c in stop: #stop word removing
    dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace(' '+c+' ', ' '))


In [17]:
print(dataframe['label'])

0       0
1       0
2       1
3       0
4       0
       ..
6995    1
6996    1
6997    0
6998    0
6999    0
Name: label, Length: 7000, dtype: int64


In [18]:
comments_attack['label'][990]==0

True

In [19]:
print(dataframe['comment'])

0       الحمدلله يارب فوز مهم يا زمالك.. الدعم ليكم يا...
1                 فدوه يا بخت فدوه يا زمن واحد منكم يجيبه
2       يا رب يا واحد يا أحد بحق يوم الاحد ان تهلك بني...
3       هواالحرية يا وجع قلبي عليكي يا امي... الله لا ...
4                يا بكون بحياتك الأهم يا إما ما بدي أكون 
                              ...                        
6995    يا حمار ، يا جاهل ، نسبة الباطل ما بتتحسبش ، و...
6996    كل زق يا طاقية يا واطي يا حقير يا تافه يا سافل...
6997    يا كبير يا ممتع يا نجم لابد أن تعي جيدا أننا م...
6998              يا رب الاتحاد يفوز يا رب.  الاتحادالنصر
6999    يعز عليا ادخل الشارع وملاقيكش واقف مستنيني الب...
Name: comment, Length: 7000, dtype: object


In [43]:
#split the data into training and validation sets
train_x, valid_x, train_y, valid_y = train_test_split(dataframe['comment'], dataframe['label'], test_size=0.07, random_state=42)

In [44]:
print('train comments length: ',len(train_x))
print('test comments length: ',len(valid_x))

train comments length:  6509
test comments length:  491


In [87]:
# print(train_y)

In [45]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(dataframe['comment'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# ngram level tf-idf 
# tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(3,4), max_features=5000)
# tfidf_vect_ngram.fit(dataframe['comment'])
# xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
# xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(dataframe['comment'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x)



In [89]:
# load the pre-trained word-embedding vectors 
embeddings_index = {}
for i, line in enumerate(open('data/wiki-news-300d-1M.vec', encoding="utf8")):
    values = line.split()
    embeddings_index[values[0]] = np.asarray(values[1:], dtype='float32')

# create a tokenizer 
token = text.Tokenizer()
token.fit_on_texts(dataframe['comment'])
word_index = token.word_index

# convert text to sequence of tokens and pad them to ensure e
# qual length vectors 
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=70)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=70)

# create token-embedding mapping
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [46]:
def train_model(classifier, xtrain, ytrain, xvalid, yvalid): 
    # fit the training dataset on the classifier
    classifier.fit(xtrain, ytrain)
    # predict the labels on validation dataset
    predictions = classifier.predict(xvalid)     
    accuracy = metrics.accuracy_score(predictions, yvalid)
    f1score = metrics.f1_score(yvalid, predictions, average='weighted')
    return accuracy, f1score

In [26]:
# # Naive Bayes on Word Level TF IDF Vectors
# accuracy, f1score = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf, valid_y)
# print("NB, WordLevel TF-IDF:   accuracy: %s     f1 score: %s"% (accuracy,f1score))

# # Naive Bayes on Ngram Level TF IDF Vectors
# accuracy, f1score = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram, valid_y)
# print("NB, N-Gram Vectors:   accuracy: %s     f1 score: %s"% (accuracy,f1score))

# # Naive Bayes on Character Level TF IDF Vectors
# accuracy, f1score = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars, valid_y)
# print("NB, CharLevel Vectors:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

In [47]:
# Linear Classifier on Word Level TF IDF Vectors
accuracy, f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf, valid_y)
print("LR, WordLevel TF-IDF:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

# # Linear Classifier on Ngram Level TF IDF Vectors
# accuracy, f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram, valid_y)
# print("LR, N-Gram Vectors:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

# Linear Classifier on Character Level TF IDF Vectors
accuracy, f1score = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars, valid_y)
print("LR, CharLevel Vectors:   accuracy: %s   f1 score: %s"% (accuracy,f1score))

LR, WordLevel TF-IDF:   accuracy: 0.8879837067209776   f1 score: 0.8659927079956297
LR, CharLevel Vectors:   accuracy: 0.8879837067209776   f1 score: 0.8730579604407931


C:\Users\mjahan18\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [78]:
# convert to one_hot
train_y_onehot = keras.utils.to_categorical(train_y, 3)
valid_y_onehot = keras.utils.to_categorical(valid_y, 3)

In [79]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, valid_y)

In [54]:
# RF on Count Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_count, train_y, xvalid_count)
print(accuracy)

# RF on Word Level TF IDF Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf, train_y, xvalid_tfidf)
print(accuracy)

# RF on Word Level TF IDF Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)

print(accuracy)



C:\Users\mjahan18\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.8128571428571428


C:\Users\mjahan18\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.8828571428571429


C:\Users\mjahan18\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.8471428571428572


In [80]:
clf = Pipeline([
    ('vect',tfidf_vect_ngram_chars),
    ('clf', linear_model.LogisticRegression()),
])

clf = clf.fit(train_x, train_y)

C:\Users\mjahan18\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [81]:
Labels=[]
IDE=[]
for i in range(len(twee)):
    IDE.append(ids[i])
    predictions = clf.predict([twee[i]])
    if predictions[0]==1:
        Labels.append('OFF')
    elif predictions[0]==0:
        Labels.append('NOT')
print(len(Labels))
print(len(IDE))

2000
2000


In [82]:
my_dict = { 'q_a' : IDE,'label' : Labels}
df = pd.DataFrame(my_dict)
df.to_csv('submission_arabic',index=False)

In [127]:
df_csv = pd.read_csv('submission_arabic.csv', sep=',')
df_csv2 = pd.read_csv('submission_arabic_2.csv',sep=',')

In [128]:
#marge all data from files that has been read

q_a=[]
label=[]
for i in range(len(df_csv)):
    label.append(df_csv['q_a'][i])
    q_a.append(df_csv['label'][i])
x=0
for i in q_a:
    if i=='OFF':
        x= x+1
print(x)
        
#marge all data from files that has been read

q_a=[]
label=[]
for i in range(len(df_csv2)):
    label.append(df_csv2['q_a'][i])
    q_a.append(df_csv2['label'][i])
x=0
for i in q_a:
    if i=='OFF':
        x= x+1
print(x)
    

168
